In [57]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import datetime

In [58]:
# Membaca dataset dari file CSV
dataset = pd.read_csv('kadaluarsa2.csv')

# Mengonversi data menjadi vektor fitur dan label
items, expiry = dataset['food'], dataset['exp_date']
expiry = np.array(expiry)

# Membuat objek Tokenizer dan mempelajari tokenisasi dari data
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(items)

print(dataset.head())
print(dataset.columns)

       food  exp_date
0    Apples         5
1   Bananas         3
2   Berries         1
3  Broccoli         3
4   Carrots         7
Index(['food', 'exp_date'], dtype='object')


In [59]:
# Mengonversi teks menjadi urutan angka berdasarkan token yang telah dipelajari
items = tokenizer.texts_to_sequences(items)

# Mengambil daftar kata yang ada dalam tokenizer
word_index = tokenizer.word_index

# Mengonversi urutan angka menjadi vektor one-hot encoding
items = tf.keras.preprocessing.sequence.pad_sequences(items)

In [60]:
# Normalisasi data
expiry = expiry / np.max(dataset['exp_date'])

In [61]:
# Split data menjadi data latih dan data uji
train_size = int(0.8 * len(dataset))
train_items, test_items = items[:train_size], items[train_size:]
train_expiry, test_expiry = expiry[:train_size], expiry[train_size:]

In [62]:
# Definisikan model menggunakan TensorFlow
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(word_index) + 1, 16, input_length=items.shape[1]),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 2, 16)             448       
                                                                 
 global_average_pooling1d_3   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_6 (Dense)             (None, 16)                272       
                                                                 
 dense_7 (Dense)             (None, 1)                 17        
                                                                 
Total params: 737
Trainable params: 737
Non-trainable params: 0
_________________________________________________________________


In [63]:
# Latih model
model.fit(train_items, train_expiry, epochs=10, batch_size=16, validation_data=(test_items, test_expiry))

Epoch 1/10
2/2 [==============================] - 3s 398ms/step - loss: 0.6926 - accuracy: 0.0500 - val_loss: 0.6913 - val_accuracy: 0.0000e+00
Epoch 2/10
2/2 [==============================] - 0s 88ms/step - loss: 0.6895 - accuracy: 0.0500 - val_loss: 0.6896 - val_accuracy: 0.0000e+00
Epoch 3/10
2/2 [==============================] - 0s 80ms/step - loss: 0.6863 - accuracy: 0.0000e+00 - val_loss: 0.6881 - val_accuracy: 0.0000e+00
Epoch 4/10
2/2 [==============================] - 0s 71ms/step - loss: 0.6832 - accuracy: 0.0000e+00 - val_loss: 0.6864 - val_accuracy: 0.0000e+00
Epoch 5/10
2/2 [==============================] - 0s 65ms/step - loss: 0.6801 - accuracy: 0.0000e+00 - val_loss: 0.6847 - val_accuracy: 0.0000e+00
Epoch 6/10
2/2 [==============================] - 0s 75ms/step - loss: 0.6767 - accuracy: 0.0000e+00 - val_loss: 0.6829 - val_accuracy: 0.0000e+00
Epoch 7/10
2/2 [==============================] - 0s 125ms/step - loss: 0.6734 - accuracy: 0.0000e+00 - val_loss: 0.6809 - va

In [66]:
# Prediksi kadaluarsa
item_names = ['Apples', 'Bananas', 'Corn']  # Masukkan item baru
purchase_dates = ['2023-05-28', '1945-05-28', '1945-05-29']  # Masukkan tanggal pembelian
current_date = datetime.date.today()

days_since_purchase = [(current_date - datetime.datetime.strptime(date, '%Y-%m-%d').date()).days
                       for date in purchase_dates]

input_items = tokenizer.texts_to_sequences(item_names)
input_items = tf.keras.preprocessing.sequence.pad_sequences(input_items, maxlen=items.shape[1])
predictions = model.predict(input_items)

1/1 [==============================] - 0s 65ms/step


In [67]:
# Cetak hasil prediksi
for item_name, prediction, days_since_purchase in zip(item_names, predictions, days_since_purchase):
    expiry_date = current_date + datetime.timedelta(days=days_since_purchase)
    if days_since_purchase <= prediction[0]*np.max(dataset['exp_date']):
        expired = 'Belum Kadaluarsa'
    else:
        expired = 'Kadaluarsa'
    print(f'{item_name}: {expired} (Kadaluarsa pada: {expiry_date.strftime("%Y-%m-%d")})')


Apples: Kadaluarsa (Kadaluarsa pada: 2101-05-30)
Bananas: Kadaluarsa (Kadaluarsa pada: 2101-05-30)
Corn: Kadaluarsa (Kadaluarsa pada: 2101-05-29)
